In [26]:
import numpy as np
import cv2
import os

def create_result_folder(base_path):
    result_folder = os.path.join(base_path, 'result')
    if not os.path.exists(result_folder):
        os.makedirs(result_folder)
    return result_folder

def img_pyramid(img_path, result_folder):
    image = cv2.imread(img_path)
    lower_reso = cv2.pyrDown(image)
    higher_reso = cv2.pyrUp(image)
    base_name = os.path.basename(img_path).split('.')[0]
    cv2.imwrite(f'{result_folder}/{base_name}_lower_reso.png', lower_reso)
    cv2.imwrite(f'{result_folder}/{base_name}_higher_reso.png', higher_reso)

def color_space_conversion(img_path, result_folder):
    image = cv2.imread(img_path)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    base_name = os.path.basename(img_path).split('.')[0]
    cv2.imwrite(f'{result_folder}/{base_name}_hsv_image.png', hsv_image)

def contrast_adjustment(img_path, result_folder):
    image = cv2.imread(img_path, 0)
    equalized_image = cv2.equalizeHist(image)
    base_name = os.path.basename(img_path).split('.')[0]
    cv2.imwrite(f'{result_folder}/{base_name}_equalized_image.png', equalized_image)

def Canny_Edge(img_path, result_folder):
    image = cv2.imread(img_path)
    edges = cv2.Canny(image, 100, 200)
    base_name = os.path.basename(img_path).split('.')[0]
    cv2.imwrite(f'{result_folder}/{base_name}_edges.png', edges)

def Morphological_Operations(img_path, result_folder):
    image = cv2.imread(img_path)
    kernel = np.ones((5, 5), np.uint8)
    dilation = cv2.dilate(image, kernel, iterations=1)
    erosion = cv2.erode(image, kernel, iterations=1)
    base_name = os.path.basename(img_path).split('.')[0]
    cv2.imwrite(f'{result_folder}/{base_name}_dilation.png', dilation)
    cv2.imwrite(f'{result_folder}/{base_name}_erosion.png', erosion)

def image_binarization(img_path, result_folder):
    image = cv2.imread(img_path)
    _, binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    base_name = os.path.basename(img_path).split('.')[0]
    cv2.imwrite(f'{result_folder}/{base_name}_binary_image.png', binary_image)

def CLAHE(img_path, result_folder):
    image = cv2.imread(img_path, 0)  # CLAHE는 흑백 이미지에서 작동하므로 0으로 읽어야 합니다.
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    equalized_image = clahe.apply(image)
    base_name = os.path.basename(img_path).split('.')[0]
    cv2.imwrite(f'{result_folder}/{base_name}_clahe.png', equalized_image)

def homomorphic_filter(img_path, result_folder, d0=30, c=1.0, gH=2.0, gL=0.5):
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # 그레이스케일로 이미지 읽기
    if image is None:
        raise ValueError("Image not found or the path is incorrect")

    # Convert image to log domain
    image_log = np.log1p(np.array(image, dtype='float32') / 255)

    # Perform DFT
    dft = cv2.dft(image_log, flags=cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)

    # Create a high pass filter
    rows, cols = image.shape
    crow, ccol = rows // 2, cols // 2
    mask = np.zeros((rows, cols, 2), np.float32)
    for u in range(rows):
        for v in range(cols):
            duv = np.sqrt((u - crow) ** 2 + (v - ccol) ** 2)
            mask[u, v] = (gH - gL) * (1 - np.exp(-c * (duv ** 2 / d0 ** 2))) + gL

    # Apply mask and inverse DFT
    fshift = dft_shift * mask
    f_ishift = np.fft.ifftshift(fshift)
    img_back = cv2.idft(f_ishift)
    img_back = cv2.magnitude(img_back[:, :, 0], img_back[:, :, 1])

    # Convert back to spatial domain
    img_homomorphic = np.expm1(img_back)

    # Clip the values to prevent overflow and normalize
    img_homomorphic = np.clip(img_homomorphic, 0, None)
    img_homomorphic = (img_homomorphic - np.amin(img_homomorphic)) / (np.amax(img_homomorphic) - np.amin(img_homomorphic))
    img_homomorphic = (img_homomorphic * 255).astype(np.uint8)

    base_name = os.path.basename(img_path).split('.')[0]
    cv2.imwrite(f'{result_folder}/{base_name}_homomorphic_image.png', img_homomorphic)

def add_lighting_effect(img_path, result_folder, center=None, radius=100, strength=1.5):
    image = cv2.imread(img_path)
    if center is None:
        center = (image.shape[1] // 2, image.shape[0] // 2)

    mask = np.zeros_like(image, dtype=np.float32)
    cv2.circle(mask, center, radius, (255, 255, 255), -1)
    mask = cv2.GaussianBlur(mask, (21, 21), 0)

    highlighted = cv2.addWeighted(image, 1, mask.astype(image.dtype), strength, 0)
    base_name = os.path.basename(img_path).split('.')[0]
    cv2.imwrite(f'{result_folder}/{base_name}_highlighted_image.png', highlighted)

def Retinex_Algorithm(img_path, result_folder):
    image = cv2.imread(img_path)
    image = image / 255.0
    sigma = 15
    retinex_image = np.log10(image + 1) - np.log10(cv2.GaussianBlur(image, (0, 0), sigma) + 1)
    retinex_image = (retinex_image - np.min(retinex_image)) / (np.max(retinex_image) - np.min(retinex_image))
    retinex_image = (retinex_image * 255).astype(np.uint8)
    base_name = os.path.basename(img_path).split('.')[0]
    cv2.imwrite(f'{result_folder}/{base_name}_retinex_image.png', retinex_image)

def image_augmentation(img_path, result_folder):
    img_pyramid(img_path, result_folder)
    color_space_conversion(img_path, result_folder)
    contrast_adjustment(img_path, result_folder)
    Canny_Edge(img_path, result_folder)
    Morphological_Operations(img_path, result_folder)
    image_binarization(img_path, result_folder)
    add_lighting_effect(img_path, result_folder)
    Retinex_Algorithm(img_path, result_folder)

def load_images_and_augmentation(folder):
    result_folder = create_result_folder(folder)
    i = 1
    for filename in os.listdir(folder):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            img_path = os.path.join(folder, filename)
            print(f'{i}th image augmentation')
            image_augmentation(img_path, result_folder)
            i += 1

# 사용자가 제공한 폴더 경로
folder_path = '/content/gdrive/MyDrive/yolo_test/image_augumentation_test/ink_4-1'

# 이미지 파일을 읽고 증강 작업을 수행합니다.
load_images_and_augmentation(folder_path)


1th image augmentation
2th image augmentation
3th image augmentation
